<a href="https://colab.research.google.com/github/harshrupendrasingh/crop_prediction/blob/main/predict2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
data_path = "/content/combined_agriculture_data.csv"
df = pd.read_csv(data_path)

print("Columns in DataFrame:", df.columns)


Columns in DataFrame: Index(['Date', 'Commodity', 'Location', 'Daily Price', 'Unit', 'Sowing Date',
       'Harvest Date', 'Production (Tons)', 'Rainfall (mm)', 'Max Temp',
       'Min Temp', 'Humidity (%)', 'Extreme Event', 'Festival/Holiday',
       'Commodity Relevance', 'Sentiment Score', 'Key Event'],
      dtype='object')


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Sowing Date'] = pd.to_datetime(df['Sowing Date'], errors='coerce')
df['Harvest Date'] = pd.to_datetime(df['Harvest Date'], errors='coerce')

target = df['Daily Price']
features = df.drop(columns=['Daily Price'])

categorical_columns = ['Commodity', 'Location', 'Unit', 'Extreme Event', 'Festival/Holiday', 'Commodity Relevance', 'Sentiment Score', 'Key Event']
date_columns = ['Date', 'Sowing Date', 'Harvest Date']

In [ ]:

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Production (Tons)', 'Rainfall (mm)', 'Max Temp', 'Min Temp', 'Humidity (%)']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough'
)

features_processed = preprocessor.fit_transform(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_processed, target, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 1
model = FeedForwardNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 14110.3398
Epoch [20/100], Loss: 14054.0986
Epoch [30/100], Loss: 13985.9697
Epoch [40/100], Loss: 13900.1299
Epoch [50/100], Loss: 13791.2910
Epoch [60/100], Loss: 13655.0654
Epoch [70/100], Loss: 13487.9658
Epoch [80/100], Loss: 13287.7734
Epoch [90/100], Loss: 13053.4297
Epoch [100/100], Loss: 12784.8721


In [ ]:
def predict(input_data):
    model.eval()
    with torch.no_grad():
        input_df = pd.DataFrame([input_data], columns=['Date', 'Commodity', 'Location', 'Unit', 'Sowing Date', 'Harvest Date', 'Production (Tons)', 'Rainfall (mm)', 'Max Temp', 'Min Temp', 'Humidity (%)', 'Extreme Event', 'Festival/Holiday', 'Commodity Relevance', 'Sentiment Score', 'Key Event'])
        input_df['Date'] = pd.to_datetime(input_df['Date'], errors='coerce')
        input_df['Sowing Date'] = pd.to_datetime(input_df['Sowing Date'], errors='coerce')
        input_df['Harvest Date'] = pd.to_datetime(input_df['Harvest Date'], errors='coerce')

        input_features = input_df.drop(columns=date_columns)
        input_processed = preprocessor.transform(input_features)
        input_tensor = torch.tensor(input_processed, dtype=torch.float32)

        predictions = model(input_tensor)
        return predictions.numpy()

In [ ]:
input_data = {
    'Date': '2022-04-11',
    'Commodity': 'gram',
    'Location': 'Mumbai',
    'Unit': 'kg',
    'Sowing Date': '2015-05-14',
    'Harvest Date': '2015-09-09',
    'Production (Tons)': 20580.85,
    'Rainfall (mm)': 141.61,
    'Max Temp': 26.23,
    'Min Temp': 22.52,
    'Humidity (%)': 54.22,
    'Extreme Event': 'None',
    'Festival/Holiday': 'Holi',
    'Commodity Relevance': 'gram',
    'Sentiment Score': 'Neutral',
    'Key Event': 'Government Policy'
}

In [ ]:
prediction = predict(input_data)
print("Prediction:", prediction)

Prediction: [[5.297532]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
